In [1]:
import pandas as pd

#### Dataset: https://www.kaggle.com/datasets/saurabhshahane/ecommerce-text-classification

In [2]:
df = pd.read_csv('ecommerce_dataset.csv',names=['category','desription'],header=None)

In [3]:
df.head()

,category,desription
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [4]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8671
Name: count, dtype: int64

We have total 4 categories
- Household
- Electronics
- Clothing and Accessories
- Books

In [5]:
df.shape

(50425, 2)

In [6]:
# drop null values
df.dropna(inplace=True)

In [7]:
#Pre process category name for access fastttext

In [8]:
df.category.replace('Clothing & Accessories','Clothing_Accessories' , inplace= True)

In [9]:
df.category.value_counts()

category
Household               19313
Books                   11820
Electronics             10621
Clothing_Accessories     8670
Name: count, dtype: int64

 When you train a fasttext model, it expects <b>labels </b> to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [10]:
df['category'] = '__label__' + df['category'].astype(str)

In [11]:
df.head()

,category,desription
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [13]:
df['cat_desc'] = df['category']+ ' '+df['desription']

In [14]:
df.head()

,category,desription,cat_desc
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


### Preprocessing
- Remove punctuation
- Remove extra space
- Make the entire sentence lower case

In [16]:
df.cat_desc[0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [17]:
import re

In [18]:
def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ',text)
    text = re.sub(r' +',' ',text)
    return text.strip().lower()
    

In [19]:
preprocess('Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7)')

'household paper plane design framed wall hanging motivational office decor art prints 8 7'

In [20]:
df['cat_desc'] = df['cat_desc'].map(preprocess)

### Train test split

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
train,test= train_test_split(df,test_size=0.2)

In [24]:
train.shape

(40339, 3)

In [25]:
test.shape

(10085, 3)

In [26]:
import fasttext

In [27]:
train.to_csv('ecommerce.train', columns=['cat_desc'],index=False,header=None)

In [28]:
test.to_csv('ecommerce.test',columns=['cat_desc'],index=False, header=False)

### Train the model and evaluate performance



In [29]:
model = fasttext.train_supervised(input='ecommerce.train')

Read 4M words
Number of words:  79369
Number of labels: 4
Progress: 100.0% words/sec/thread: 6605172 lr:  0.000000 avg.loss:  0.194128 ETA:   0h 0m 0s


In [30]:
model.test('ecommerce.test')

(10083, 0.9693543588217792, 0.9693543588217792)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively.

### Prediction

In [34]:
model.predict('wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3')

(('__label__electronics',), array([0.99838167]))

In [35]:
model.predict('THE LOVE HYPOTHESS BY ALI HAZELWOOD')

(('__label__books',), array([1.00001001]))

In [36]:
model.predict('midrise cotton pant')

(('__label__clothing_accessories',), array([1.0000062]))